## Original T5

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0:00:00


In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [5]:
torch_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [6]:
torch_device

'cpu'

In [ ]:
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)
# torch_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
text = 'I has had a problem with the model. Maybe you help?'
print(correct_grammar(text, num_return_sequences=1))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['the model. Maybe you help?']


In [ ]:
text = "Although I've known him for a while, I still can't believe how much stubborn he is"
print(correct_grammar(text, num_return_sequences=1))

["I've known him for a while, but I can't believe how much stubborn he is."]


## Trained T5

In [7]:
model_name = '/content/drive/MyDrive/11611/project/checkpoint-128000_11_17'
torch_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text], truncation=True, padding='max_length', max_length=512, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=512,num_beams=3, num_return_sequences=num_return_sequences, do_sample=True, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [109]:
def correct_grammar_top_p(input_text,num_return_sequences):
  batch = tokenizer([input_text], truncation=True, padding='max_length', max_length=512, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=512, num_return_sequences=num_return_sequences, do_sample=True, temperature=1.5 ,top_p=0.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [22]:
ßtext = 'She no went to the market.'
print(correct_grammar(text, num_return_sequences=3))

['She went to the market.', 'She went to the market.', 'She never went to the market.']


In [58]:
text = 'She no went to the market.'
print(correct_grammar_top_p(text, num_return_sequences=6))

['She went to the market.', 'She never went to the market.', 'She no went to the market.', 'She no longer went to the market.', 'She went to the market.', 'She went to the market.']


In [26]:
text = "We discussed about the issue"
print(correct_grammar(text, num_return_sequences=3))

['We discussed the issue', 'We discussed the issue', 'We discussed the issue before']


In [59]:
text = "We discussed about the issue"
print(correct_grammar_top_p(text, num_return_sequences=3))

['We discussed the issue', 'We discussed the issue.', 'We discussed the issue']


In [28]:
text = "I think smoke should to be ban in all restarants."
print(correct_grammar(text, num_return_sequences=3))

['I think smoke should be banned in all restarants.', 'I think smoke should be banned in all areas.', 'I think smoke should be banned in all restarants.']


In [60]:
text = "I think smoke should to be ban in all restarants."
print(correct_grammar_top_p(text, num_return_sequences=3))

['I think smoke should be banned in all areas.', 'I think smoke should be banned in all areas.', 'I think smoke should be banned in all other restaurants.']


In [62]:
text = 'There is not no a doubt tracking system have bring many benefit at this information age.'
print(correct_grammar(text, num_return_sequences=3))

['There is not a doubt that tracking systems can bring many benefits at this information age.', 'There is not a doubt that tracking systems can bring many benefits at this time.', 'There is not a doubt that tracking systems can bring many benefits at this time of year.']


In [63]:
text = 'There is not no a doubt tracking system have bring many benefit at this information age.'
print(correct_grammar_top_p(text, num_return_sequences=3))

['There is not a doubt that tracking systems have brought many benefits at this age.', 'There is no doubt the tracking system could bring many benefits at this age.', 'There is no doubt tracking systems have brought many benefits at this information age.']


In [64]:
text = "On one hand, we do not want this potential danger causing firghtenning affects in our families' later lives."
print(correct_grammar(text, num_return_sequences=3))

["On one hand, we do not want this potential danger causing any affects in our families' later lives.", "On one hand, we do not want this potential danger causing problems in our families' later lives.", "On one hand, we do not want this potential danger causing any affects in our families' future lives."]


In [ ]:
sequence = "Although I've known him for a while , I still can't believe how much stubborn he is ."

In [ ]:
tokenizer.tokenize(sequence)

['▁Although',
 '▁I',
 "'",
 've',
 '▁known',
 '▁him',
 '▁for',
 '▁',
 'a',
 '▁while',
 '▁',
 ',',
 '▁I',
 '▁still',
 '▁can',
 "'",
 't',
 '▁believe',
 '▁how',
 '▁much',
 '▁stubborn',
 '▁',
 'he',
 '▁is',
 '▁',
 '.']

In [65]:
text = 'I has had a problem with the model. Maybe you help?'
print(correct_grammar(text, num_return_sequences=3))

['I have had a problem with the model. Maybe you can help?', 'I have had a problem with the model. Maybe you help?', 'I have had a problem with the model. Maybe you could help?']


In [66]:
text = 'I has had a problem with the model. Maybe you help?'
print(correct_grammar_top_p(text, num_return_sequences=3))

['I have had a problem with the model. Maybe you can help?', 'I have had a problem with the model. Maybe you help?', 'I have had a problem with the model. Maybe you can help?']


In [ ]:
text = "Although I've known him for a while, I still can't believe how much stubborn he is."
print(correct_grammar(text, num_return_sequences=1))

["Although I've known him for a while, I still can't believe how stubborn he is."]


In [68]:
text = "Although I've known him for a while, I still can't believe how much stubborn he is."
print(correct_grammar_top_p(text, num_return_sequences=3))

["Although I've known him for a while, I still can't believe how stubborn he is.", "Although I've known him for a while, I still can't believe how stubborn he is.", "Although I've known him for a while, I still can't believe how stubborn he is."]


In [69]:
text = "Jeff ran a mile and drops his keys. "
print(correct_grammar(text, num_return_sequences=3))

['Jeff ran a mile and dropped his keys.', 'Jeff ran a mile and dropped his keys.', 'Jeff ran a mile and slipped his keys.']


In [71]:
text = "Jeff runs a mile and dropped his keys. "
print(correct_grammar_top_p(text, num_return_sequences=5))

['Jeff ran a mile and dropped his keys.', 'Jeff ran a mile and dropped his keys.', 'Jeff runs a mile and dropped his keys.', 'Jeff runs a mile and dropped his keys.', 'Jeff runs for a mile and drops his keys.']


In [72]:
text = "I walk to the store and I bought milk."
print(correct_grammar(text, num_return_sequences=3))

['I went to the store and I bought milk.', 'I walked to the store and I bought milk.', 'I went to the store and I bought the milk.']


In [73]:
text = "I walk to the store and I bought milk."
print(correct_grammar_top_k(text, num_return_sequences=3))

['I went to the store and I bought milk.', 'I walk to the store and I bought milk.', 'I went to the store and I bought milk.']


In [45]:
text = "what be the reason for everyone leave the company."
print(correct_grammar(text, num_return_sequences=3))

['what is the reason for everyone to leave the company.', 'what is the reason for everyone leaving the company.', 'what is the reason for everyone leaving the company?']


In [49]:
text = "what be the reason for everyone leave the company."
print(correct_grammar_top_k(text, num_return_sequences=5))

['what was the reason for everyone to leave the company.', 'what is the reason for everyone leaving the company.', 'what would be the reason for everyone to leave the company.', 'what would be the reason for everyone to leave the company.', 'what would be the reason for everyone to leave the company.']


In [47]:
text = "Anna and Mike is going skiing."
print(correct_grammar(text, num_return_sequences=3))

['Anna and Mike are going skiing.', 'Anna and Mike are going skiing.', 'Anna and Mike are going skiing.']


In [74]:
text = "Anna and Mike is going skiing."
print(correct_grammar_top_p(text, num_return_sequences=5))

['Anna and Mike are going skiing.', 'Anna and Mike are going skiing.', 'Anna and Mike are going skiing.', 'Anna and Mike are going skiing.', 'Anna and Mike are going skiing.']


In [51]:
text = "How is they?"
print(correct_grammar(text, num_return_sequences=3))

['How are they?', 'How were they?', 'How are you?']


In [75]:
text = "How is they?"
print(correct_grammar_top_p(text, num_return_sequences=5))

['How are they?', 'How are they?', 'How are they?', 'How are they?', 'How are they?']


In [76]:
text = "We enjoys horror movies"
print(correct_grammar(text, num_return_sequences=3))

['We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.']


In [78]:
text = "We enjoys horror movies"
print(correct_grammar_top_p(text, num_return_sequences=10))

['We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.', 'We enjoy horror movies.']


In [79]:
text = "She was boring in the class."
print(correct_grammar(text, num_return_sequences=3))

['She was boring in the class.', 'She was boring in the class.', 'She was boring in class.']


In [80]:
text = "She was boring in the class."
print(correct_grammar_top_p(text, num_return_sequences=3))

['She was boring in the class.', 'She was boring in class.', 'She was boring in class.']


In [81]:
text = "The life is hard !"
print(correct_grammar(text, num_return_sequences=2))

['The life is hard!', 'The life is hard!']


In [87]:
text = "The life is hard !"
print(correct_grammar_top_p(text, num_return_sequences=5))

["It's hard!", 'The life is hard!', 'It is hard to live on!', 'The life is hard!', 'The life is hard!']


In [88]:
text = "My mother wanted that I be doctor."
print(correct_grammar(text, num_return_sequences=3))

['My mother wanted to be a doctor.', 'My mother wanted me to be doctor.', 'My mother wanted to be doctor.']


In [89]:
text = "My mother wanted that I be doctor."
print(correct_grammar_top_p(text, num_return_sequences=3))

['My mother wanted me to be a doctor.', 'My mother wanted me to be doctor.', 'My mother wanted to be a doctor.']


In [94]:
text = "My brother has 10 years."
print(correct_grammar(text, num_return_sequences=3))

['My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years old.']


In [96]:
text = "My brother has 10 years."
print(correct_grammar_top_p(text, num_return_sequences=10))

['My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.', 'My brother has 10 years.']


In [97]:
text = "I want eat now ."
print(correct_grammar(text, num_return_sequences=3))

['I want to eat now.', 'I want to eat now.', 'I want to eat now.']


In [98]:
text = "My husband engineer."
print(correct_grammar(text, num_return_sequences=3))

['My husband was engineer.', 'My husband was an engineer.', 'My husband is an engineer.']


In [107]:
text = "My husband engineer."
print(correct_grammar_top_p(text, num_return_sequences=3))

['My husband is an engineer.', 'My husband engineer.', 'My husband and engineer']


In [100]:
text = "Above all, life is more important than secret."
print(correct_grammar(text, num_return_sequences=3))

['Above all, life is more important than secret.', 'Above all, life is more important than a secret.', 'Above all, life is more important than secrets.']


In [106]:
text = "Above all, life is more important than secret."
print(correct_grammar_top_p(text, num_return_sequences=10))

['Above all, life is more important than it is secret.', 'Above all, life is more important than secret.', 'Above all, life is more important than the secret.', 'Above all, life is more important than it is secret.', 'Above all, life is more important than a secret.', 'Above all, life is more important than life is secret.', 'Above all, life is more important than secret.', 'Above all, life is more important than the secret.', 'Above all, life is more important than secret.', 'Above all, life is more important than a secret.']


In [ ]:
from transformers import T5Model, T5Tokenizer, T5ForConditionalGeneration, T5Config
import torch
import torch.nn as nn

class T5Classifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(T5Classifier, self).__init__()
        self.encoder = T5Model.from_pretrained(model_name).encoder
        self.classifier_head = nn.Linear(self.encoder.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        # Get hidden states from the encoder
        encoder_output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)

        # Use the last hidden state for classification
        last_hidden_state = encoder_output.last_hidden_state[:, 0, :]

        # Classify using the classifier head
        logits = self.classifier_head(last_hidden_state)

        return logits

# Example usage
model_name = "/content/drive/MyDrive/11611/project/checkpoint-120000_11_10"
num_labels = 2  # Change this based on your classification task
tokenizer = T5Tokenizer.from_pretrained(model_name)
classifier_model = T5Classifier(model_name, num_labels)

# Input example
input_text = "Your input text here."
input_ids = tokenizer.encode(input_text, return_tensors="pt")
attention_mask = torch.ones(input_ids.shape)

# Forward pass
logits = classifier_model(input_ids, attention_mask)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
logits

tensor([[-0.0079,  0.1224]], grad_fn=<AddmmBackward0>)